In [17]:
import pandas as pd

In [22]:
dt_execute_rbg = pd.read_pickle("result/Execute RBG.pkl")
dt_start_handle_request = pd.read_pickle("result/Start Handle Request.pkl")
dt_end_handle_request = pd.read_pickle("result/End Handle Request.pkl")
dt_path_movement_finished = pd.read_pickle("result/Path Movement Finished.pkl")
dt_path_movement_finished_detail = pd.read_pickle("result/Path Movement Finished - Detail.pkl")
dt_path_movement_failed = pd.read_pickle("result/Path Movement Failed.pkl")
dt_unknown = pd.read_pickle("result/Unknown Content.pkl")
dt_alerts_handled = pd.read_pickle("result/Alerts Handled.pkl")
dt_alerts_processing = pd.read_pickle("result/Alerts Processing.pkl")
dt_alerts_received = pd.read_pickle("result/Alerts Received.pkl")
dt_telegrams_processed = pd.read_pickle("result/Telegrams Processed.pkl")
dt_telegrams_sent = pd.read_pickle("result/Telegrams Sent.pkl")
dt_is_position_ok = pd.read_pickle("result/Check Position.pkl")
dt_check_sequence = pd.read_pickle("result/Check Sequence.pkl")
dt_is_vb_ok = pd.read_pickle("result/IsVBOK.pkl")


In [24]:
dt_execute_rbg

,timestamp,info_code,thread,operation_num,stage,task_id,task_type,category,locked,points,telegram_type,lam,info,le,fail_status
0,2025.02.06 07:00:00.091,I,MF2,G42036843,try execute Task:,3887,SingleTask,ON_TIME,false,1372,FS,1,SingleTask - LAM(s)={1},1847105,None
1,2025.02.06 07:00:00.294,I,MF2,G42036843,try execute Task:,3882,SingleTask,ON_TIME,false,1352,FS,1,SingleTask - LAM(s)={1},1847079,None
2,2025.02.06 07:00:00.497,I,MF2,G42036843,try execute Task:,3891,SingleTask,ON_TIME,false,1342,FS,1,SingleTask - LAM(s)={1},1847015,None
3,2025.02.06 07:00:00.700,I,MF2,G42036843,try execute Task:,3899,SingleTask,ON_TIME,false,1292,FS,1,SingleTask - LAM(s)={1},1847053,None
4,2025.02.06 07:00:00.903,I,MF2,G42036843,try execute Task:,3890,SingleTask,ON_TIME,false,1292,FS,1,SingleTask - LAM(s)={1},1847111,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137947,2025.02.06 11:59:57.828,I,MF2,G42762253,try execute Task:,169,SingleTask,ON_TIME,false,1440,FS,1,SingleTask - LAM(s)={1},1847755,None
137948,2025.02.06 11:59:58.750,I,MF2,G42762253,try execute Task:,180,SingleTask,ON_TIME,false,1422,FS,1,SingleTask - LAM(s)={1},1847673,None
137949,2025.02.06 11:59:59.156,I,MF2,G42762253,try execute Task:,175,SingleTask,ON_TIME,false,1420,FS,1,SingleTask - LAM(s)={1},1847783,None
137950,2025.02.06 11:59:59.580,I,MF2,G42762253,try execute Task:,176,SingleTask,ON_TIME,false,1418,FS,1,SingleTask - LAM(s)={1},1847784,None


In [20]:
cols_with_mainLoop = [
    col 
    for col in dt_unknown.columns 
    if dt_unknown[col].astype(str).str.contains("Alert-Thread", na=False).any()
]
print(cols_with_mainLoop)

[]


In [21]:
unique_stages = dt_unknown['stage'].unique()
print(unique_stages)

['cleanMfsTrans' 'sendAvAlert_SND_LHM_EINGELAGERT' 'sendAVImpl'
 'sendRBGFS' 'updateFilled' 'updatePlanned' 'info' 'sendLhmFachtiefe'
 'checkCPU' 'mfsFachtiefe' 'sendRBGGB' 'sendLhmAusgelagert'
 'setStatusAnyWarte' 'sendFMDO' 'sendFMTA' 'cleanup' 'cleanMfsTransLog'
 'cleanRbgLog' 'cleanAuftrag' 'cleanSeqLog' 'cleanLogFm' 'cleanLogRbg'
 'cleanLogRbgEr' 'cleanLogRbgZs' 'cleanLogAv' 'cleanMfsTransErrTxt'
 'cleanBetriebsmittelLog' 'cleanMfsTransportStats' 'cleanNaviActive'
 'cleanLogRbgBck' 'cleanMfsTransLogBck' 'cleanAuftragsende' 'mfsLoeschen'
 'alert:MF1_FROMTSS' 'processAlertQueue'
 'sendAvAlert_SND_LHM_ANGEKOMMEN_LONG' 'mfsStart' 'mfsStart[laenge'
 'mfsStart[breite' 'mfsStart[hoehe' 'sendAvAlert_SND_LHM_ANGEKOMMEN_KURZ'
 'mfsEinlagern' 'mfsEinlagern[laenge' 'mfsEinlagern[breite'
 'mfsEinlagern[hoehe' 'sendFMLZ' 'updateLocks'
 'sendAvAlert_SND_LHM_VORBEIGEFAHREN' 'sendAvAlert_SND_LHM_ABGENOMMEN'
 'fa' 'getAndSetNioDestination' 'sendLhmWirdAusgeschleust' 'mfsAuslagern'
 'mfsAuslagern[pe